### example of a mutual exclusion (mutex) lock
From
https://superfastpython.com/thread-mutex-lock/

In [22]:
from time import sleep
from random import random
from threading import Thread
from threading import Lock
 
# work function
def task(lock, identifier, value):
    # acquire the lock
    with lock:
        print(f'>thread {identifier} got the lock, sleeping for {value}')
        sleep(value)
 
# create a shared lock
lock = Lock()
# start a few threads that attempt to execute the same critical section
for i in range(10):
    # start a thread
    Thread(target=task, args=(lock, i, random()*10)).start()
# wait for all threads to finish...

>thread 0 got the lock, sleeping for 4.081728085537989
>thread 1 got the lock, sleeping for 2.9080556068569843
>thread 2 got the lock, sleeping for 0.38023598304484096
>thread 3 got the lock, sleeping for 8.550443591198139
>thread 4 got the lock, sleeping for 1.0953735605563364
>thread 5 got the lock, sleeping for 3.508343621315876
>thread 6 got the lock, sleeping for 1.0787949290262078
>thread 7 got the lock, sleeping for 7.704572791740967
>thread 8 got the lock, sleeping for 9.276338005581165
>thread 9 got the lock, sleeping for 4.831648940122299


### Sin Lock

In [23]:
from time import sleep
from random import random
from threading import Thread
from threading import Lock
j=0
# work function
def task(identifier, value):
        global j
        sleep(value)
        print(f'>thread {identifier} got the lock, sleeping for {value}')
        j=identifier
        

# start a few threads that attempt to execute the same critical section
buffer=[]
for i in range(10):
    # start a thread
    buffer.append(Thread(target=task, args=(i, random()*10)))
    
[ii.start() for ii in buffer]
[ii.join() for ii in buffer]
#print(len(buffer))#[8].join()    
print(j)

>thread 0 got the lock, sleeping for 1.157563827916207
>thread 8 got the lock, sleeping for 2.647398564148956
>thread 7 got the lock, sleeping for 4.679344639329992
>thread 4 got the lock, sleeping for 5.06987670252851
>thread 1 got the lock, sleeping for 5.959680197684204
>thread 9 got the lock, sleeping for 8.015499356504698
>thread 5 got the lock, sleeping for 8.216977552791665
>thread 2 got the lock, sleeping for 8.59617630685347
>thread 3 got the lock, sleeping for 9.023270643600767
>thread 6 got the lock, sleeping for 9.6133690948969
6


Otra forma de Lock ... (para toda la tarea)

In [26]:
from time import sleep
from random import random
from threading import Thread
from threading import Lock
j=0
# work function
def task(identifier, value):
        global j
        sleep(value)
        print(f'>thread {identifier} got the lock, sleeping for {value}')
        j=identifier
        

# start a few threads that attempt to execute the same critical section
buffer=[]
for i in range(10):
    # start a thread
    buffer.append(Thread(target=task, args=(i, random()*10)))
    buffer[i].start()
    buffer[i].join()
#print(len(buffer))#[8].join()    
print(j)

>thread 0 got the lock, sleeping for 0.04626775918289194
>thread 1 got the lock, sleeping for 0.27269618925416794
>thread 2 got the lock, sleeping for 2.3092868050205673
>thread 3 got the lock, sleeping for 4.1951113635331625
>thread 4 got the lock, sleeping for 2.5069325378410587
>thread 5 got the lock, sleeping for 8.264430840635494
>thread 6 got the lock, sleeping for 3.788899875612075
>thread 7 got the lock, sleeping for 2.7114486633257293
>thread 8 got the lock, sleeping for 2.348366396356253
>thread 9 got the lock, sleeping for 2.9215369925531176
9


python-ipc-examples/process_to_process-with-fifo.py

From

https://github.com/spurin/python-ipc-examples/tree/master

In [ ]:
import logging
import os
import time
from multiprocessing import Process

# Process1 logic
def process1():
    process1_logger = logging.getLogger('process1')
    process1_logger.info(f"Pid:{os.getpid()}")
    fifo = '/tmp/process_fifo.txt'

    # Create a fifo, os.mkfifo will block until there is a reader (process2)
    os.mkfifo(fifo)

    # Open fifo for writing
    file = open(fifo, 'w')

    # Write 10 entries
    for i in range(1,11):

        # Attempt to write to our fifo until succession
        while True:
            try:
                process1_logger.info(f"Writing {int(i)}")
                file.write(f"{i}\n")
                file.flush()
                break
            except:
                pass

    # Clean up fifo
    file.close()

    # Grace for the read process to complete
    process1_logger.info("Sleeping for 2")
    time.sleep(2)

    # Log completion
    process1_logger.info("Finished process 1")


# Process2 logic
def process2():
    process2_logger = logging.getLogger('process2')
    process2_logger.info(f"Pid:{os.getpid()}")
    fifo = '/tmp/process_fifo.txt'

    # Keep attempting to open the fifo, ignore race condition failures
    while True:
        try:
            file = open(fifo, 'r')
            break
        except:
            pass

    # Expect 10 entries
    count = 0
    while count < 10:
        while True:
            try:
                line = file.readline()
                process2_logger.info(f"Read: {int(line)}")
                count += 1
                break
            except:
                pass

    # Clean up fifo
    file.close()
    os.remove(fifo)

    # Log completion
    process2_logger.info("Finished process 2")


# Main
def main():

    # Setup parent logger and log pid
    parent_logger = logging.getLogger('parent')
    parent_logger.info(f"Pid:{os.getpid()}")

    # Setup processes
    procs = [Process(target=process1), Process(target=process2)]

    # Start processes
    for proc in procs:
        proc.start()

    # Run to completion
    for proc in procs:
        proc.join()

# Setup simple logging
logging.basicConfig(level=logging.INFO)

# Execute main
if __name__ == '__main__':
    main()

python-ipc-examples/process_to_process-with-pipe

In [ ]:
#!/usr/bin/env python3

import logging
import os
import time
import multiprocessing
from multiprocessing import Process

# Process1 logic
def process1(pipe):
    process1_logger = logging.getLogger('process1')
    process1_logger.info(f"Pid:{os.getpid()}")

    # Open the file descriptor
    file = os.fdopen(pipe.fileno(), 'w')
    process1_logger.info("Opened file descriptor")

    # Write 10 entries
    for i in range(1,11):

        # Attempt to write to our pipe until succession
        while True:
            try:
                process1_logger.info(f"Writing {int(i)}")
                file.write(f"{i}\n")
                file.flush()
                if i % 6 == 0:
                    process1_logger.info("Intentionally sleeping for 5 seconds")
                    time.sleep(5)
                break
            except:
                pass

    # Clean up pipe
    pipe.close()

    # Log completion
    process1_logger.info("Finished process 1")


# Process2 logic
def process2(pipe):
    process2_logger = logging.getLogger('process2')
    process2_logger.info(f"Pid:{os.getpid()}")

    # Open the file descriptor
    file = os.fdopen(pipe.fileno(), 'r')
    process2_logger.info("Opened file descriptor")

    # Expect 10 entries
    count = 0
    while count < 10:
        while True:
            try:
                line = file.readline()
                process2_logger.info(f"Read: {int(line)}")
                count += 1
                break
            except Exception:
                pass

    # Clean up pipe
    pipe.close()

    # Log completion
    process2_logger.info("Finished process 2")


# Main
def main():

    # Setup parent logger and log pid
    parent_logger = logging.getLogger('parent')
    parent_logger.info(f"Pid:{os.getpid()}")

    # Setup pipe
    r, w = multiprocessing.Pipe(False)

    # Setup processes
    procs = [Process(target=process1, args=(w,)), Process(target=process2, args=(r,))]

    # Start processes
    for proc in procs:
        proc.start()

    # Run to completion
    for proc in procs:
        proc.join()

# Setup simple logging
logging.basicConfig(level=logging.INFO)

# Execute main
if __name__ == '__main__':
    main()

python-ipc-examples/process_to_process-with-pipe



In [ ]:
#!/usr/bin/env python3

import logging
import os
import time
from multiprocessing import Process, Array
# On MacPro 5,1, module gave Illegal instruction: 4
# Installing non-wheel version with non-wheel dependencies
# in clean venv works
#
# pip install zerorpc --no-binary :all:
import zerorpc

# Process1 logic
def process1():
    process1_logger = logging.getLogger('process1')
    process1_logger.info(f"Pid:{os.getpid()}")

    class RPC(object):
        def __init__(self):
            self.value = 0

        def get_item(self):
            self.value += 1
            process1_logger.info(f"Writing {self.value}")
            return self.value

    # Setup an RPC Server
    s = zerorpc.Server(RPC())
    s.bind("tcp://0.0.0.0:4242")

    # Run the server without background'ing for 5 seconds
    # (s.run() will block)
    zerorpc.gevent.spawn(s.run)
    zerorpc.gevent.sleep(5)

    # Log completion
    process1_logger.info("Finished process 1")


# Process2 logic
def process2():
    process2_logger = logging.getLogger('process2')
    process2_logger.info(f"Pid:{os.getpid()}")

    c = zerorpc.Client()
    c.connect("tcp://127.0.0.1:4242")

    # Expect 10 entries
    for i in range(10):
        while True:
            try:
                line = c.get_item()
                process2_logger.info(f"Read: {int(line)}")
                break
            except Exception:
                pass

    # Log completion
    process2_logger.info("Finished process 2")


# Main
def main():

    # Setup parent logger and log pid
    parent_logger = logging.getLogger('parent')
    parent_logger.info(f"Pid:{os.getpid()}")

    # Setup shared memory using Array (multiprocessing)
    arr = Array('i', [-1] * 10)

    # Setup processes
    procs = [Process(target=process1), Process(target=process2)]

    # Start processes
    for proc in procs:
        proc.start()

    # Run to completion
    for proc in procs:
        proc.join()

# Setup simple logging
logging.basicConfig(level=logging.INFO)

# Execute main
if __name__ == '__main__':
    main()

python-ipc-examples/process_to_process-with-sharedmemory

In [ ]:
#!/usr/bin/env python3

import logging
import os
import time
from multiprocessing import Process, Array

# Process1 logic
def process1(shared):
    process1_logger = logging.getLogger('process1')
    process1_logger.info(f"Pid:{os.getpid()}")

    # Write 10 entries
    for i in range(1,11):

        # Attempt to write to our shared memory until succession
        while True:
            try:
                process1_logger.info(f"Writing {int(i)}")
                shared[i-1] = i
                if i % 6 == 0:
                    process1_logger.info("Intentionally sleeping for 5 seconds")
                    time.sleep(5)
                break
            except Exception as e:
                print(str(e))
                pass

    # Log completion
    process1_logger.info("Finished process 1")


# Process2 logic
def process2(shared):
    process2_logger = logging.getLogger('process2')
    process2_logger.info(f"Pid:{os.getpid()}")

    # Expect 10 entries
    for i in range(10):
        while True:
            try:
                line = shared[i]
                if line == -1:
                    process2_logger.info("Data not available sleeping for 1 second before retrying")
                    time.sleep(1)
                    raise Exception('pending')
                process2_logger.info(f"Read: {int(line)}")
                break
            except Exception:
                pass

    # Log completion
    process2_logger.info("Finished process 2")


# Main
def main():

    # Setup parent logger and log pid
    parent_logger = logging.getLogger('parent')
    parent_logger.info(f"Pid:{os.getpid()}")

    # Setup shared memory using Array (multiprocessing)
    arr = Array('i', [-1] * 10)

    # Setup processes
    procs = [Process(target=process1, args=(arr,)), Process(target=process2, args=(arr,))]

    # Start processes
    for proc in procs:
        proc.start()

    # Run to completion
    for proc in procs:
        proc.join()

# Setup simple logging
logging.basicConfig(level=logging.INFO)

# Execute main
if __name__ == '__main__':
    main()